### Importing Libraries

In [1]:
#!pip install imblearn
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN

#### Reading csv

In [4]:
df=pd.read_csv("tel_churn.csv")
df.head()

Unnamed: 0  SeniorCitizen  MonthlyCharges  TotalCharges  Churn  \
0           0              0           29.85         29.85      0   
1           1              0           56.95       1889.50      0   
2           2              0           53.85        108.15      1   
3           3              0           42.30       1840.75      0   
4           4              0           70.70        151.65      1   

   gender_Female  gender_Male  Partner_No  Partner_Yes  Dependents_No  ...  \
0              1            0           0            1              1  ...   
1              0            1           1            0              1  ...   
2              0            1           1            0              1  ...   
3              0            1           1            0              1  ...   
4              1            0           1            0              1  ...   

   PaymentMethod_Bank transfer (automatic)  \
0                                        0   
1                                        0   
2                                        0   
3                                        1   
4                                        0   

   PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
0                                      0                               1   
1                                      0                               0   
2                                      0                               0   
3                                      0                               0   
4                                      0                               1   

   PaymentMethod_Mailed check  tenure_group_1 - 12  tenure_group_13 - 24  \
0                           0                    1                     0   
1                           1                    0                     0   
2                           1                    1                     0   
3                           0                    0                     0   
4                           0                    1                     0   

   tenure_group_25 - 36  tenure_group_37 - 48  tenure_group_49 - 60  \
0                     0                     0                     0   
1                     1                     0                     0   
2                     0                     0                     0   
3                     0                     1                     0   
4                     0                     0                     0   

   tenure_group_61 - 72  
0                     0  
1                     0  
2                     0  
3                     0  
4                     0  

[5 rows x 52 columns]

In [5]:
df=df.drop('Unnamed: 0',axis=1)

In [9]:
x=df.drop('Churn',axis=1)
x

SeniorCitizen  MonthlyCharges  TotalCharges  gender_Female  gender_Male  \
0                 0           29.85         29.85              1            0   
1                 0           56.95       1889.50              0            1   
2                 0           53.85        108.15              0            1   
3                 0           42.30       1840.75              0            1   
4                 0           70.70        151.65              1            0   
...             ...             ...           ...            ...          ...   
7027              0           84.80       1990.50              0            1   
7028              0          103.20       7362.90              1            0   
7029              0           29.60        346.45              1            0   
7030              1           74.40        306.60              0            1   
7031              0          105.65       6844.50              0            1   

      Partner_No  Partner_Yes  Dependents_No  Dependents_Yes  PhoneService_No  \
0              0            1              1               0                1   
1              1            0              1               0                0   
2              1            0              1               0                0   
3              1            0              1               0                1   
4              1            0              1               0                0   
...          ...          ...            ...             ...              ...   
7027           0            1              0               1                0   
7028           0            1              0               1                0   
7029           0            1              0               1                1   
7030           0            1              1               0                0   
7031           1            0              1               0                0   

      ...  PaymentMethod_Bank transfer (automatic)  \
0     ...                                        0   
1     ...                                        0   
2     ...                                        0   
3     ...                                        1   
4     ...                                        0   
...   ...                                      ...   
7027  ...                                        0   
7028  ...                                        0   
7029  ...                                        0   
7030  ...                                        0   
7031  ...                                        1   

      PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
0                                         0                               1   
1                                         0                               0   
2                                         0                               0   
3                                         0                               0   
4                                         0                               1   
...                                     ...                             ...   
7027                                      0                               0   
7028                                      1                               0   
7029                                      0                               1   
7030                                      0                               0   
7031                                      0                               0   

      PaymentMethod_Mailed check  tenure_group_1 - 12  tenure_group_13 - 24  \
0                              0                    1                     0   
1                              1                    0                     0   
2                              1                    1                     0   
3                              0                    0                     0   
4                              0                    1              

In [10]:
y=df['Churn']
y

0       0
1       0
2       1
3       0
4       1
       ..
7027    0
7028    0
7029    0
7030    1
7031    0
Name: Churn, Length: 7032, dtype: int64

##### Train Test Split

In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

#### Decision Tree Classifier

In [12]:
model_dt=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [13]:
model_dt.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [14]:
y_pred=model_dt.predict(x_test)
y_pred

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [15]:
model_dt.score(x_test,y_test)

0.8052594171997157

In [19]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87      1046
           1       0.65      0.53      0.58       361

    accuracy                           0.81      1407
   macro avg       0.75      0.71      0.73      1407
weighted avg       0.80      0.81      0.80      1407



###### As you can see that the accuracy is quite low, and as it's an imbalanced dataset, we shouldn't consider Accuracy as our metrics to measure the model, as Accuracy is cursed in imbalanced datasets.

###### Hence, we need to check recall, precision & f1 score for the minority class, and it's quite evident that the precision, recall & f1 score is too low for Class 1, i.e. churned customers.

###### Hence, moving ahead to call SMOTEENN (UpSampling + ENN)

In [21]:
from imblearn.combine import SMOTEENN
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(x, y)

In [22]:
xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)

In [23]:
model_dt_smote=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [24]:
model_dt_smote.fit(xr_train,yr_train)
yr_predict = model_dt_smote.predict(xr_test)
model_score_r = model_dt_smote.score(xr_test, yr_test)
print(model_score_r)
print(metrics.classification_report(yr_test, yr_predict))

0.9163822525597269
              precision    recall  f1-score   support

           0       0.89      0.93      0.91       529
           1       0.94      0.91      0.92       643

    accuracy                           0.92      1172
   macro avg       0.91      0.92      0.92      1172
weighted avg       0.92      0.92      0.92      1172



In [25]:
print(metrics.confusion_matrix(yr_test, yr_predict))

[[491  38]
 [ 60 583]]


###### Now we can see quite better results, i.e. Accuracy: 92 %, and a very good recall, precision & f1 score for minority class.

###### Let's try with some other classifier.

#### Random Forest Classifier

In [26]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
model_rf=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [28]:
model_rf.fit(x_train,y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [29]:
y_pred=model_rf.predict(x_test)

In [30]:
model_rf.score(x_test,y_test)

0.8137882018479033

In [31]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.83      0.94      0.88      1046
           1       0.72      0.45      0.56       361

    accuracy                           0.81      1407
   macro avg       0.77      0.70      0.72      1407
weighted avg       0.80      0.81      0.80      1407



In [34]:
from imblearn.combine import SMOTEENN

sm = SMOTEENN()
X_resampled1, y_resampled1 = sm.fit_resample(x, y)

In [35]:
xr_train1,xr_test1,yr_train1,yr_test1=train_test_split(X_resampled1, y_resampled1,test_size=0.2)

In [36]:
model_rf_smote=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [37]:
model_rf_smote.fit(xr_train1,yr_train1)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [38]:
yr_predict1 = model_rf_smote.predict(xr_test1)

In [39]:
model_score_r1 = model_rf_smote.score(xr_test1, yr_test1)

In [40]:
print(model_score_r1)
print(metrics.classification_report(yr_test1, yr_predict1))

0.9512820512820512
              precision    recall  f1-score   support

           0       0.96      0.93      0.95       564
           1       0.94      0.97      0.95       606

    accuracy                           0.95      1170
   macro avg       0.95      0.95      0.95      1170
weighted avg       0.95      0.95      0.95      1170



In [41]:
print(metrics.confusion_matrix(yr_test1, yr_predict1))

[[527  37]
 [ 20 586]]


###### With RF Classifier, also we are able to get quite good results, infact better than Decision Tree.

###### We can now further go ahead and create multiple classifiers to see how the model performance is, but that's not covered here, so you can do it by yourself :)

#### Performing PCA

In [42]:
# Applying PCA
from sklearn.decomposition import PCA
pca = PCA(0.9)
xr_train_pca = pca.fit_transform(xr_train1)
xr_test_pca = pca.transform(xr_test1)
explained_variance = pca.explained_variance_ratio_

In [43]:
model=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [44]:
model.fit(xr_train_pca,yr_train1)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [45]:
yr_predict_pca = model.predict(xr_test_pca)

In [46]:
model_score_r_pca = model.score(xr_test_pca, yr_test1)

In [47]:
print(model_score_r_pca)
print(metrics.classification_report(yr_test1, yr_predict_pca))

0.7256410256410256
              precision    recall  f1-score   support

           0       0.76      0.63      0.69       564
           1       0.70      0.82      0.76       606

    accuracy                           0.73      1170
   macro avg       0.73      0.72      0.72      1170
weighted avg       0.73      0.73      0.72      1170



#### Pickling the model

In [48]:
import pickle

In [49]:
filename = 'model.sav'

In [50]:
pickle.dump(model_rf_smote, open(filename, 'wb'))

In [51]:
load_model = pickle.load(open(filename, 'rb'))

In [52]:
model_score_r1 = load_model.score(xr_test1, yr_test1)

In [53]:
model_score_r1

0.9512820512820512